# HRFAV Fundus Dataset - Single Image Import

This notebook demonstrates how to import images one by one using the new `/import/image` endpoint.

For more information: https://eyened.github.io/eyened-platform/getting_started/

In [1]:
import requests
from pathlib import Path
from tqdm.notebook import tqdm
import os
import base64
from datetime import date
from utils import download_and_extract_dataset

# API configuration
IMAGES_BASEPATH="//path/to/images" # the same set in .env
API_BASE_URL = "http://<HOST>:<PORT>/api"  # Adjust this to your server URL
API_USERNAME = "admin"  # Replace with your API username
API_PASSWORD = "CHANGE_ME"  # Replace with your API password

# Create authentication header
auth_str = f"{API_USERNAME}:{API_PASSWORD}"
auth_bytes = auth_str.encode('ascii')
base64_auth = base64.b64encode(auth_bytes).decode('ascii')
headers = {
    "Authorization": f"Basic {base64_auth}",
    "Content-Type": "application/json"
}

In [2]:
# Download HRF dataset if not already downloaded
extract_dir = Path(IMAGES_BASEPATH) / "hrfav"
if not extract_dir.exists():
    os.makedirs(extract_dir, exist_ok=True)
    download_and_extract_dataset('https://www5.cs.fau.de/fileadmin/research/datasets/fundus-images/all.zip', extract_dir)

PermissionError: [Errno 13] Permission denied: '//path/to/images'

In [3]:
# Set up project information
project_name = "HRFAV Fundus Dataset - One by One"

In [4]:
# Get all image paths
images_dir = extract_dir / "images"
image_paths = list(images_dir.glob("*.jpg")) + list(images_dir.glob("*.JPG"))

print(f"Found {len(image_paths)} images.")

Found 45 images.


## Importing Images One by One

Unlike the bulk import approach, we'll now import each image individually using the `/import/image` endpoint.

First, we'll create a project structure where each image gets its own patient. Each image will be assigned a patient ID based on the image name.

In [5]:
# Define import options - we need to create patients, studies, and project
import_options = {
    "create_patients": True,
    "create_studies": True, 
    "create_series": True,
    "create_project": True,
    "include_stack_trace": True
}

# For tracking results
import_results = []

In [ ]:
# Import each image individually
for idx, img_path in enumerate(tqdm(image_paths, desc="Importing images")):
    # Create unique patient and series identifiers based on the image name
    patient_id = f"Patient_{idx//2}"
    
    # Create image data structure for a single image
    image_data = {
        "project_name": project_name,
        "patient_identifier": patient_id,
        "study_date": date.today().isoformat(),
        "image": str(img_path.relative_to(IMAGES_BASEPATH)),
        "image_props": {
            "OldPath": img_path.stem,
            "Laterality": "R" if idx % 2 == 0 else "L"  # Alternating laterality for demo
        }
    }
    
    # Create the payload
    payload = {
        "data": image_data,
        "options": import_options
    }
    
    # Call the image import endpoint
    response = requests.post(
        f"{API_BASE_URL}/import/image",
        headers=headers,
        json=payload
    )
    
    result = response.json()
    import_results.append(result)
    
    # If an error occurred, print it but continue with the next image
    if not result["success"]:
        print(f"Error importing {img_path.name}: {result['error']}")
        print(result['stack_trace'])
        break

Importing images:   0%|          | 0/45 [00:00<?, ?it/s]

In [7]:
# Summarize the import results
successful_imports = [r for r in import_results if r["success"]]
failed_imports = [r for r in import_results if not r["success"]]

print(f"Total images: {len(image_paths)}")
print(f"Successfully imported: {len(successful_imports)}")
print(f"Failed imports: {len(failed_imports)}")

if failed_imports:
    print("\nFailed imports:")
    for fail in failed_imports:
        print(f"  {fail['image_path']}: {fail['error']}")

Total images: 45
Successfully imported: 45
Failed imports: 0
